[Use Case](https://github.com/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb)

[Querying Retriever](https://github.com/sugarforever/LangChain-Advanced/blob/main/Retrievers/05_SelfQuerying_Retriever/05_Self_Querying_Retriever.ipynb)

In [6]:
import autogen
import os
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-2.0-flash"],
    },
)

with open('gemini_key.txt', 'r') as f:
    gemini_key = f.read().strip()
os.environ["GOOGLE_API_KEY"] = gemini_key

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
import pandas as pd
from langchain.schema import Document

df = pd.read_csv('train.csv')

docs = [
    Document(page_content=f"Domanda: {row['Sentence']}\nRisposta: {row['Response']}")
    for _, row in df.iterrows()
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
vectorstore.add_documents(docs)

C:\Users\Susanna\AppData\Local\Temp\ipykernel_2632\3944536505.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


['e5a453a9-719b-4a34-a048-41784b1f4a1a',
 '66972faf-bc08-49d9-bbb5-937204a6d205',
 '37e8d7fa-9d8d-42c2-b841-558fdb67345f',
 '4f4e498c-f5c3-4f8c-b2eb-b3b2474714f1',
 'c262b992-dab5-4193-9ec1-e9b46303c9f3',
 '0e8ec4f2-12b2-4d24-80ad-959ab1f0b322',
 'f1389ea6-dd8f-410b-a236-dc5293f629f5',
 '37ab5ca9-82b9-4dac-8fb0-a9f4ead68db3',
 '6cde8d07-c370-4260-92a3-056c62042c7b',
 '50db86a5-214e-4404-bd7f-3087c1d79864',
 '81400e41-f55e-441c-a27e-d4517b37f516',
 '94fede32-23eb-483d-9c44-9f1ee063e7da',
 '66c43a06-72de-49e5-b6ba-b15580e6ae89',
 '07c96636-d977-43ec-860c-d3a4dea02b72',
 '82172e07-239a-4d68-b5b1-5dec2eb1192d',
 '5bdc0bdb-26f5-41c2-8ab8-05bfc61e7cee',
 '1b7e9644-5fcc-4bf3-8ccb-8ba9aebca58f',
 'd5ee7b4a-2956-4c86-9905-a4f177459116',
 'fed5fd3a-89c4-41ec-ab8d-5d79772019fa',
 '4e865a9d-68b0-4fd4-9f42-0b07613f3c92',
 '2253cdf9-e1c3-4d7d-9d80-4e2fb0c852f1',
 'ad6bc7c9-46c1-49d4-9e2b-e58b41d33ff0',
 '7e8be295-edd3-4874-8d5d-2a011a041c48',
 '7c35d927-2732-4992-8db2-3068b5291434',
 '4df1c68c-1ccd-

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

qa = ConversationalRetrievalChain.from_llm(
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    retriever=vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    #chain_type_kwargs={"prompt": prompt_template}
)

In [13]:
result = qa(({"question": "Come l’asma può influenzare la mia vita?"}))
result['answer']

"L'asma può determinare mancanza di respiro e rendere difficile l'esecuzione delle attività quotidiane"

In [17]:
def answer_question(question):
  response = qa({"question": question})
  return response["answer"]

llm_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.1,
    "functions": [
        {
            "name": "answer_question",
            "description": "Answer any questions related to Hypertension and symptoms.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to answer",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [18]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

import os
os.environ["AUTOGEN_USE_DOCKER"] = "False"
user_proxy = autogen.UserProxyAgent("user", code_execution_config=False)

In [19]:
res = assistant.initiate_chat(user_proxy, message="Come posso aiutarti oggi?")
print(assistant)

assistant (to user):

Come posso aiutarti oggi?

--------------------------------------------------------------------------------
user (to assistant):

Come l’asma può influenzare la mia vita?

--------------------------------------------------------------------------------
assistant (to user):

L'asma può influenzare la tua vita in diversi modi, sia fisici che emotivi. Ecco alcuni esempi:

**Effetti fisici:**

*   **Difficoltà respiratorie:** Questo è l'effetto più ovvio. L'asma può causare respiro corto, affanno, tosse e senso di oppressione al petto. Questi sintomi possono rendere difficile svolgere attività quotidiane come camminare, salire le scale, fare esercizio fisico o persino parlare.
*   **Attacchi d'asma:** Gli attacchi d'asma possono essere spaventosi e potenzialmente pericolosi per la vita. Possono richiedere cure mediche immediate, inclusi farmaci di emergenza o persino il ricovero in ospedale.
*   **Disturbi del sonno:** La tosse e il respiro sibilante notturni possono 